In [1]:
import os
#os.environ["NCCL_DEBUG"] = "INFO"

import sys
sys.path.append("..")

#import jax
#jax.config.update("jax_enable_x64", True)

from utils import *
from model_tn import *
from keras_utils import *
from jax_utils import *

keras.mixed_precision.set_global_policy("mixed_float16")
#keras.mixed_precision.set_global_policy("float32")
#keras.mixed_precision.set_global_policy("float64")

In [2]:
model = EfficientPhys()
model.compile(optimizer='adamw', loss='mse')
model(np.random.random((4, 160, 72, 72, 3)));
model.summary()

Model: "efficient_phys"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ tsm (TSM)                       │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (640, 72, 72, 32)      │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (640, 70, 70, 32)      │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (640, 35, 35, 64)      │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (640, 33, 33, 64)      │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (640, 70, 70, 32)      │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_mask (Attention_mask) │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (640, 33, 33, 64)      │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (640, 35, 35, 32)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (640, 35, 35, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (640, 128)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (640, 128)             │     2,097,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (640, 1)               │           129 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (640, 5184, 3)         │        20,736 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (640, 16384)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,188,929 (8.35 MB)

 Trainable params: 2,178,561 (8.31 MB)

 Non-trainable params: 10,368 (40.50 KB)

In [3]:
@jax.jit
@partial(shard_map, mesh=mesh, in_specs=P('x'), out_specs=P('x'))
def diff_label(x, y):
    y = ops.concatenate([y[:,1:]-y[:,:-1], ops.zeros_like(y[:,:1])], axis=1)
    return x, y

batch_size = 32

rlap_tape = "/root/ssd_cache/rppg_training_data/rlap_160x72x72_all"

train_rlap = load_datatape(rlap_tape, fold='train', batch=batch_size)
valid_rlap = load_datatape(rlap_tape, fold='val', extended_hr='False', batch=batch_size)

train_rlap = DatatapeMonitor(train_rlap)
training_set, validation_set = KerasDataset(train_rlap), KerasDataset(valid_rlap)

training_set = training_set.apply_fn(compress_aug)
training_set = training_set.apply_fn(diff_label)
validation_set = validation_set.apply_fn(diff_label)

rlap_160x72x72_all             datatape has been loaded.    24704 items total. fold=train selected
rlap_160x72x72_all             datatape has been loaded.     3040 items total. fold=val&extended_hr=False selected


In [4]:
#model.fit(training_set, validation_data=validation_set, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/efficientphys.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)])
train(model, training_set, validation_set, epochs=20, check_point_path='../weights/efficientphys.weights.h5')
model.load_weights('../weights/efficientphys.weights.h5')

Training:   0%|          | 0/772 [00:00<?, ?it/s]

HTML(value='')

Validating:   0%|          | 0/95 [00:00<?, ?it/s]

Epoch   1:  Training loss:0.1574	Validation loss:0.1322	


Training:   0%|          | 0/772 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/95 [00:00<?, ?it/s]

Epoch  19:  Training loss:0.1379	Validation loss: 0.122	


Training:   0%|          | 0/772 [00:00<?, ?it/s]

Validating:   0%|          | 0/95 [00:00<?, ?it/s]

Epoch  20:  Training loss:0.1378	Validation loss:0.1226	


In [5]:
eval_on_dataset(dataset_H5_mmpd, pmodel(model), 160, (72, 72), step=4, batch=16, save='../results/EfficientPhys_RLAP_MMPD.h5', cumsum=True)
get_metrics('../results/EfficientPhys_RLAP_MMPD.h5', dropped='False')

  0%|          | 0/660 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,13.163±0.189,20.772±3.164,15.379±0.223,0.31407
Whole,9.342±0.408,14.025±3.935,10.754±0.457,0.53484


In [6]:
eval_on_dataset(dataset_H5_cohface, pmodel(model), 160, (72, 72), step=4, batch=16, save='../results/EfficientPhys_RLAP_COHFACE.h5', cumsum=True, fps=30)
get_metrics('../results/EfficientPhys_RLAP_COHFACE.h5')

  0%|          | 0/164 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,5.711±0.232,11.435±3.167,8.188±0.344,0.57136
Whole,4.342±0.407,6.784±2.872,6.266±0.6,0.80694


In [7]:
eval_on_dataset(dataset_H5_pure, pmodel(model), 160, (72, 72), step=4, batch=16, save='../results/EfficientPhys_RLAP_PURE.h5', cumsum=True)
get_metrics('../results/EfficientPhys_RLAP_PURE.h5')

  0%|          | 0/59 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,2.245±0.28,8.04±3.599,3.883±0.527,0.94018
Whole,1.684±0.453,3.868±2.456,3.016±0.887,0.98793


In [8]:
eval_on_dataset(dataset_H5_ubfc_rppg2, pmodel(model), 160, (72, 72), step=4, batch=16, save='../results/EfficientPhys_RLAP_UBFC.h5', cumsum=True)
get_metrics('../results/EfficientPhys_RLAP_UBFC.h5')

  0%|          | 0/42 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,1.664±0.183,4.505±3.087,1.695±0.163,0.96453
Whole,0.83±0.236,1.738±1.506,0.825±0.22,0.99483
